In [1]:
from preparation import Preprocessing,FeatureEngineering,import_data
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import metrics


In [2]:
raw_data = pd.read_csv(r'data\data.csv', delimiter=';')

In [3]:
pipeline = Pipeline([
    ('Feature_Engineering', FeatureEngineering()),
    ('Preprocessing', Preprocessing())
])

In [4]:
X,y = pipeline.fit_transform(raw_data.drop('y',axis=1), raw_data['y'])
X,y

(            age  campaign     pdays  previous  emp.var.rate  cons.price.idx  \
 0      1.533034 -0.565922 -0.170834 -0.349494      0.648092        0.722722   
 1      1.628993 -0.565922 -0.170834 -0.349494      0.648092        0.722722   
 2     -0.290186 -0.565922 -0.170834 -0.349494      0.648092        0.722722   
 3     -0.002309 -0.565922 -0.170834 -0.349494      0.648092        0.722722   
 4      1.533034 -0.565922 -0.170834 -0.349494      0.648092        0.722722   
 ...         ...       ...       ...       ...           ...             ...   
 41183  3.164336 -0.565922 -0.170834 -0.349494     -0.752343        2.058168   
 41184  0.573445 -0.565922 -0.170834 -0.349494     -0.752343        2.058168   
 41185  1.533034 -0.204909 -0.170834 -0.349494     -0.752343        2.058168   
 41186  0.381527 -0.565922 -0.170834 -0.349494     -0.752343        2.058168   
 41187  3.260295  0.156105 -0.170834  1.671136     -0.752343        2.058168   
 
        cons.conf.idx  euribor3m  nr.e

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Perform Grid Search with Cross-Validation
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=cv, scoring='roc_auc')
grid_search_rf.fit(X_train, y_train)

# Print the best score and parameters
print(grid_search_rf.best_score_)
print(grid_search_rf.best_params_)

KeyboardInterrupt: 

In [ ]:
best_rf_model = grid_search_rf.best_estimator_
best_rf_model.fit(X_train, y_train)
y_pred_rf =best_rf_model.predict(X_test)

In [ ]:
precision = metrics.precision_score(y_test, y_pred_rf)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred_rf).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

# Extreme Gradient Boosting Classifier

In [ ]:
import xgboost as xgb

param_grid = {
    "max_depth": [2, 4, 6],  # Largeur des arbres
    "n_estimators": [25, 50, 100],  # Nombre d'arbres
    "learning_rate": [0.1, 0.2, 0.5, 0.7],  # Taux d'apprentissage
    "subsample": [0.3, 0.5, 0.7, 0.9],  # Ratio de sous-échantillonnage
}

xgb_model = xgb.XGBClassifier(eval_metric="auc")
clf = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    cv=cv,
)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
best_xgb_model = clf.best_estimator_
best_xgb_model.fit(X_train, y_train)
y_pred = best_xgb_model.predict(X_test)

In [ ]:
precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results

In [ ]:
from preparation import do_the_job
data = do_the_job(path="data/data.csv")
X = data['input']
Y = data['output']

X.describe()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Extreme Gradient Boosting Classifier

In [ ]:
import xgboost as xgb

param_grid = {
    "max_depth": [2, 4, 6],  # Largeur des arbres
    "n_estimators": [25, 50, 100],  # Nombre d'arbres
    "learning_rate": [0.1, 0.2, 0.5, 0.7],  # Taux d'apprentissage
    "subsample": [0.3, 0.5, 0.7, 0.9],  # Ratio de sous-échantillonnage
}

xgb_model = xgb.XGBClassifier(eval_metric="auc")
clf = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    cv=cv,
)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
best_xgb_model = clf.best_estimator_
best_xgb_model.fit(X_train, y_train)
y_pred = best_xgb_model.predict(X_test)

In [ ]:
precision = metrics.precision_score(y_test, y_pred)
print(f"Précision : {precision}")

tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
results = {
    'True': [tp, fn],
    'False': [fp, tn]
}
results = pd.DataFrame(results, index=['Predicted True', 'Predicted False'])
results